# Shoreline Monitor Future Positions

Future Shoreline Positions Vousdoukas 2021

Notebook environment to migrate netcdf files to CF compliant zarr

In [38]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

### Configure OS independent paths

In [39]:
# Import standard packages
import os
import pathlib

import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import math

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))
sys.path.append(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\coclicodata')

# Get root paths
home = pathlib.Path().home()
root = home.root

# Import custom functionality
from etl import p_drive
from etl.CF_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
gca_data_dir = p_drive.joinpath("1000545-054-globalbeaches", "15_GlobalCoastalAtlas", "datasets")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    home.joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

<IPython.core.display.Javascript object>

In [40]:
# Project paths & files (manual input)
dataset_dir = gca_data_dir.joinpath("03_Shorelinemonitor_future")
dataset_dir_shorelinemonitor_fut = dataset_dir.joinpath("shorelinemonitor_future.nc")
dataset_out_file = "ShorelineMonitor_Future"
CF_dir = gca_data_dir.joinpath(r"CF")  # directory to save output CF check files

<IPython.core.display.Javascript object>

In [41]:
dataset_dir_shorelinemonitor_fut

WindowsPath('P:/1000545-054-globalbeaches/15_GlobalCoastalAtlas/datasets/03_Shorelinemonitor_future/shorelinemonitor_future.nc')

<IPython.core.display.Javascript object>

In [42]:
# # # # write csv to netcdf

# # # # Load data from CSV file into a pandas dataframe
# csv_dir = r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\03_Shorelinemonitor_future\future_shorelines.csv'
# df_fut = pd.read_csv(csv_dir)

# # Convert the pandas dataframe to an xarray dataset
# ds = xr.Dataset.from_dataframe(df_fut)

# # Write the xarray dataset to a netCDF file
# ds.to_netcdf(r'C:\Users\heijd_de\shorelinemonitor_fut.nc')

# df_fut.head(2)

<IPython.core.display.Javascript object>

### Check CF compliancy original NetCDF files

In [43]:
# open datasets
dataset_shorelinemonitor_fut = xr.open_dataset(r'C:\Users\heijd_de\shorelinemonitor_future.nc')

# check original dataset
dataset_shorelinemonitor_fut

<xarray.Dataset>
Dimensions:        (index: 696994)
Coordinates:
  * index          (index) int64 0 1 2 3 4 ... 696990 696991 696992 696993
Data variables: (12/19)
    transect_id    (index) object ...
    country_id     (index) object ...
    continent      (index) object ...
    country_name   (index) object ...
    changerate     (index) float64 ...
    intercept      (index) float64 ...
    ...             ...
    RCP4.5_p5      (index) object ...
    RCP4.5_p50     (index) object ...
    RCP4.5_p95     (index) object ...
    RCP8.5_p5      (index) object ...
    RCP8.5_p50     (index) object ...
    RCP8.5_p95     (index) object ...

<IPython.core.display.Javascript object>

In [44]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=dataset_dir_shorelinemonitor_fut, 
                 working_dir=CF_dir
                 )


<IPython.core.display.Javascript object>

In [45]:
# save original CF compliancy
save_compliancy(cap, testfile=dataset_dir_shorelinemonitor_fut, working_dir=CF_dir)

{'P:\\1000545-054-globalbeaches\\15_GlobalCoastalAtlas\\datasets\\03_Shorelinemonitor_future\\shorelinemonitor_future.nc': {'warnings': '39', 'errors': '11'}}


<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [46]:
import json

# NetCDF attribute alterations
f_global = open(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\03_Shorelinemonitor_future\metadata_shorelinemonitor_future.json')
meta_global = json.load(f_global)

for attr_name, attr_val in meta_global.items():
    if attr_name == 'PROVIDERS':
        attr_val = json.dumps(attr_val)
    dataset_shorelinemonitor_fut.attrs[attr_name] = attr_val

dataset_shorelinemonitor_fut.attrs['Conventions'] = "CF-1.8"

<IPython.core.display.Javascript object>

In [47]:
dataset_shorelinemonitor_fut

<xarray.Dataset>
Dimensions:        (index: 696994)
Coordinates:
  * index          (index) int64 0 1 2 3 4 ... 696990 696991 696992 696993
Data variables: (12/19)
    transect_id    (index) object ...
    country_id     (index) object ...
    continent      (index) object ...
    country_name   (index) object ...
    changerate     (index) float64 ...
    intercept      (index) float64 ...
    ...             ...
    RCP4.5_p5      (index) object ...
    RCP4.5_p50     (index) object ...
    RCP4.5_p95     (index) object ...
    RCP8.5_p5      (index) object ...
    RCP8.5_p50     (index) object ...
    RCP8.5_p95     (index) object ...
Attributes: (12/21)
    TITLE:               The Shoreline Monitor Future Projections
    TITLE_ABBREVIATION:  shore_mon_fut
    DESCRIPTION:         The Shoreline Monitor Future Projections is a datase...
    SHORT_DESCRIPTION:   The Shoreline Monitor Future Projections is a datase...
    INSTITUTION:         Deltares
    PROVIDERS:           {"name": "Deltares", "url": "www.deltares.nl", "role...
    ...                  ...
    DOI:                 https://doi.org/10.1038/s41558-020-0697-0
    LONG_NAME:           THE_SHORELINE_MONITOR_FUT
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [48]:
# drop time xarray

# dataset_shorelinemonitor = dataset_shorelinemonitor.drop_vars('dt')
# TODO: this needs to be changes as start_dates are different now and not always at 1984-01

date_range = [pd.to_datetime(x) for x in ['1984-01-01', '2021-01-01', '2050-01-01', '2021-01-01']]
dataset_shorelinemonitor_fut['time_step'] = xr.DataArray(date_range, dims = ['time']) #TODO: altered time_range into time_step

<IPython.core.display.Javascript object>

In [49]:
#combine start and end coordinates into a transect
from shapely.geometry import LineString

start_lons = dataset_shorelinemonitor_fut['Start_lon'].values
start_lats = dataset_shorelinemonitor_fut['Start_lat'].values
end_lons = dataset_shorelinemonitor_fut['End_lon'].values
end_lats = dataset_shorelinemonitor_fut['End_lat'].values
coords = zip(zip(start_lons,start_lats), zip(end_lons,end_lats))

dataset_shorelinemonitor_fut['transect_geom'] = (['index'], [str(LineString(line)) for line in coords])
dataset_shorelinemonitor_fut['transect_geom'].attrs['long_name'] = 'Transect Geometry'

<IPython.core.display.Javascript object>

In [51]:
# NetCDF variable and dimension alterations

# rename or swap dimension names, the latter in case the name already exists as coordinate


f_vars = open(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\03_Shorelinemonitor_future\vars_shorelinemonitor_fut.json')
meta_vars = json.load(f_vars)

for var_name, var_dict in meta_vars.items():
    dataset_shorelinemonitor_fut = dataset_shorelinemonitor_fut.rename_vars({var_name : var_dict['name']})
    for key, value in var_dict.items():
        if key != 'name':
            dataset_shorelinemonitor_fut[var_dict['name']].attrs[key] = value

dataset_shorelinemonitor_fut = dataset_shorelinemonitor_fut.rename_dims({"index": "stations"})

delete_vars = ['changerate', 'intercept', 'Intersect_lon', 'Intersect_lat', 'End_lon', 'End_lat']
for dv in delete_vars:
    dataset_shorelinemonitor_fut = dataset_shorelinemonitor_fut.drop(dv)
    
# change dtypes
object_vars = ['transect_id', 'country_id', 'continent', 'country']
for ov in object_vars:
    dataset_shorelinemonitor_fut[ov] = dataset_shorelinemonitor_fut[ov].astype('S')

all_vars = list(dataset_shorelinemonitor_fut.keys())
data_vars = ["sp_ambient", "sp_rcp4.5_p5", "sp_rcp4.5_p50", "sp_rcp4.5_p95", "sp_rcp8.5_p5", "sp_rcp8.5_p50", "sp_rcp8.5_p95"]
# set some data variables to coordinates to avoid duplication of dimensions in later stage
dataset_shorelinemonitor_fut = dataset_shorelinemonitor_fut.set_coords([v for v in all_vars if v not in data_vars])

# drop index xarray
dataset_shorelinemonitor_fut = dataset_shorelinemonitor_fut.drop('index')

<IPython.core.display.Javascript object>

In [56]:
sp_amb = xr.DataArray(
    np.array([json.loads(dataset_shorelinemonitor_fut['sp_ambient'].values[i]) for i in range(len(dataset_shorelinemonitor_fut['sp_ambient'].values))]),
    dims=['stations', 'time_step'])

for i in range(len(dataset_shorelinemonitor_fut['sp_rcp4.5_p5'].values)):
    try:
        json.loads(dataset_shorelinemonitor_fut['sp_rcp4.5_p5'].values[i])
    except:
        print(dataset_shorelinemonitor_fut['sp_rcp4.5_p5'].values[i])

sp_rcp45_p5 = xr.DataArray(
    np.array([json.loads(dataset_shorelinemonitor_fut['sp_rcp4.5_p5'].values[i]) for i in range(len(dataset_shorelinemonitor_fut['sp_rcp4.5_p5'].values))]),
    dims=['stations', 'time_step'])

sp_rcp45_p50 = xr.DataArray(
    np.array([json.loads(dataset_shorelinemonitor_fut['sp_rcp4.5_p50'].values[i]) for i in range(len(dataset_shorelinemonitor_fut['sp_rcp4.5_p50'].values))]),
    dims=['stations', 'time_step'])

sp_rcp45_p95 = xr.DataArray(
    np.array([json.loads(dataset_shorelinemonitor_fut['sp_rcp4.5_p95'].values[i]) for i in range(len(dataset_shorelinemonitor_fut['sp_rcp4.5_p95'].values))]),
    dims=['stations', 'time_step'])

sp_rcp85_p5 = xr.DataArray(
    np.array([json.loads(dataset_shorelinemonitor_fut['sp_rcp8.5_p5'].values[i]) for i in range(len(dataset_shorelinemonitor_fut['sp_rcp8.5_p5'].values))]),
    dims=['stations', 'time_step'])

sp_rcp85_p50 = xr.DataArray(
    np.array([json.loads(dataset_shorelinemonitor_fut['sp_rcp8.5_p50'].values[i]) for i in range(len(dataset_shorelinemonitor_fut['sp_rcp8.5_p50'].values))]),
    dims=['stations', 'time_step'])

sp_rcp85_p95 = xr.DataArray(
    np.array([json.loads(dataset_shorelinemonitor_fut['sp_rcp8.5_p95'].values[i]) for i in range(len(dataset_shorelinemonitor_fut['sp_rcp8.5_p95'].values))]),
    dims=['stations', 'time_step'])

delete_vars = ['sp_ambient', 'sp_rcp4.5_p5', 'sp_rcp4.5_p50', 'sp_rcp4.5_p95', 'sp_rcp8.5_p5', 'sp_rcp8.5_p50', 'sp_rcp8.5_p95', 'time_step']
for dv in delete_vars:
    dataset_shorelinemonitor_fut = dataset_shorelinemonitor_fut.drop(dv)

replacements = [sp_amb, sp_rcp45_p5, sp_rcp45_p50, sp_rcp45_p95, sp_rcp85_p5, sp_rcp85_p50, sp_rcp85_p95]
for repl in replacements:
    dataset_shorelinemonitor_fut = dataset_shorelinemonitor_fut.assign(repl)




[0, 10.479646534272566, nan, nan]
[0, 2.8243793323456376, nan, nan]
[0, 13.933836607565095, nan, nan]
[0, -16.227028285984403, nan, nan]
[0, -37.949729381086726, nan, nan]
[0, 299.7740476104356, nan, nan]
[0, -141.82271549999825, nan, nan]
[0, 38.71586298757242, nan, nan]
[0, 317.67371406764767, nan, nan]
[0, 436.61761007326663, nan, nan]
[0, 16.67050974894873, nan, nan]
[0, -104.6915756659262, nan, nan]
[0, -48.06862275958286, nan, nan]
[0, 3.453987986987402, nan, nan]
[0, 24.74982191119428, nan, nan]
[0, 7.472311858889652, nan, nan]
[0, 5.181213037162098, nan, nan]
[0, 6.120822198705047, nan, nan]
[0, -7.484447475051226, nan, nan]
[0, 7.970049454956087, nan, nan]
[0, 7.330431450085151, nan, nan]
[0, 17.984474471631984, nan, nan]
[0, -11.824357807256533, nan, nan]
[0, -4.509089956464261, nan, nan]
[0, 8.91900140691369, nan, nan]
[0, 0.2757822602895511, nan, nan]
[0, 126.5085238390349, nan, nan]
[0, 118.77204213150912, nan, nan]
[0, -9.910258721923512, nan, nan]
[0, 5.854613207095143, 

JSONDecodeError: Expecting value: line 1 column 25 (char 24)

<IPython.core.display.Javascript object>

In [ ]:
dataset_shorelinemonitor_fut

In [ ]:
# save new .nc files
dataset_shorelinemonitor_fut['time'].attrs.pop('units', None)
dataset_shorelinemonitor_fut.to_netcdf(path=str(dataset_dir_shorelinemonitor_fut).replace(".nc", "_CF.nc"))

: 

### Check CF compliancy altered NetCDF files

In [ ]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=str(dataset_dir_shorelinemonitor_hr).replace(".nc", "_CF.nc"), working_dir=CF_dir)

: 

In [ ]:
# save altered CF compliancy
save_compliancy(
    cap,
    testfile=str(dataset_dir_shorelinemonitor_hr).replace(".nc", "_CF.nc"),
    working_dir=CF_dir,
)

: 

### write data to Zarr files

In [ ]:
dataset_shorelinemonitor['outliers'].values[0]

: 

In [ ]:
# export to zarr in write mode (to overwrite if exists)
dataset_shorelinemonitor.to_zarr(dataset_dir.joinpath("%s.zarr" % dataset_out_file), mode="w")

: 

In [ ]:
test = xr.open_zarr(dataset_dir.joinpath("%s.zarr" % dataset_out_file))

: 

: 